In [1]:
import psycopg2
import pgvector
import pandas as pd
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="290128"
)

/var/folders/y6/npmgkj1j4mzgx950yrf632lc0000gn/T/ipykernel_993/523620502.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
conn.autocommit=True
conn 
if(False):
    cur=conn.cursor()
    cur.execute("CREATE EXTENSİON vector;")

In [3]:
MODEL_NAME='all-MiniLM-L6-v2' #Kullandığımız model cümle dönüştüren modeldir. Cümleleri ve paragrafları 384 boyutlu yoğun vektör uzayına eşler ve kümeleme veya anlamsal arama gibi görevler için kullanılabilir.
from sentence_transformers import SentenceTransformer
model=SentenceTransformer(MODEL_NAME)

/Users/gizemkaryagdi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
docs= [ 
{"input":"We formed our partnership with OpenAI around a shared ambition to \ responsibly advance cutting-edge AI research and democratize AI as a new \ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \ In this next phase of our partnership, developers and organizations across \ industries will have access to the best AI infrastructure, models, and toolchain \ with Azure to build and run their applications."}
]
print('len(docs)',len(docs))

len(docs) 1


In [5]:
documents=[]
ids=[]
for index,doc in enumerate(docs):
    doc_content=doc["input"]
    documents.append(doc_content)
    ids.append(str(index))
    embeddings=model.encode(documents)

In [6]:
embeddings[0].tolist()

[-0.026725545525550842,
 -0.054436445236206055,
 -0.047934334725141525,
 0.027443425729870796,
 0.09098034352064133,
 -0.013835313729941845,
 -0.005485240835696459,
 -0.04992423579096794,
 0.04061346501111984,
 0.009536135010421276,
 -0.061506278812885284,
 -0.0077025918290019035,
 0.032477278262376785,
 0.017658721655607224,
 0.0651795044541359,
 0.14944639801979065,
 0.008546381257474422,
 -0.029865946620702744,
 -0.09027165174484253,
 -0.010371614247560501,
 -0.037613723427057266,
 0.028216414153575897,
 -0.003459315747022629,
 -0.056050460785627365,
 0.03761869668960571,
 0.021069301292300224,
 0.011141099035739899,
 -0.06682039052248001,
 -0.0055345697328448296,
 0.0036667650565505028,
 -0.002282370114699006,
 -0.014770053327083588,
 0.05275994539260864,
 0.0360734686255455,
 0.030314330011606216,
 0.043572887778282166,
 -0.02920306846499443,
 -0.06029091775417328,
 -0.03818494454026222,
 -0.09580733627080917,
 0.059583377093076706,
 -0.025053158402442932,
 -0.02381063997745514,
 

In [7]:
size=len(embeddings[0])
size

384

In [8]:
if(False):
    cur=con.cursor()
    cur.execute("DROP TABLE IF EXISTS items;")
    cur.close()

In [9]:
if(True):
    cur=conn.cursor()
    cur.execute("CREATE TABLE items (id bigserial PRIMARY KEY, text TEXT, embedding_vector FLOAT[]);")
    cur.close()

In [10]:
cur = conn.cursor()
for index, item in enumerate(documents):
    my_doc = {"id": index, "text": documents[index], "embedding": embeddings[index].tolist()}
    cur.execute("""INSERT INTO items (id, text, embedding_vector) VALUES (%(id)s, %(text)s, %(embedding)s)""", my_doc)
conn.commit()
cur.close()

In [11]:
my_doc

{'id': 0,
 'text': 'We formed our partnership with OpenAI around a shared ambition to \\ responsibly advance cutting-edge AI research and democratize AI as a new \\ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \\ In this next phase of our partnership, developers and organizations across\xa0\\ industries will have access to the best AI infrastructure, models, and toolchain \\ with Azure to build and run their applications.',
 'embedding': [-0.026725545525550842,
  -0.054436445236206055,
  -0.047934334725141525,
  0.027443425729870796,
  0.09098034352064133,
  -0.013835313729941845,
  -0.005485240835696459,
  -0.04992423579096794,
  0.04061346501111984,
  0.009536135010421276,
  -0.061506278812885284,
  -0.0077025918290019035,
  0.032477278262376785,
  0.017658721655607224,
  0.0651795044541359,
  0.14944639801979065,
  0.008546381257474422,
  -0.029865946620702744,
  -0.09027165174484253,
  -0.010371614247560501,
  -0.037613723427057266,
  0.028216414153575897,

In [12]:
cur=conn.cursor()
cur.execute("""SELECT count(*) as cnt FROM items LIMIT 10""")
for r in cur.fetchall():
    print(r)
cur.close()

(1,)


In [13]:
cur=conn.cursor()
cur.execute("""SELECT id,text,embedding_vector FROM items LIMIT 10""")
for r in cur.fetchall():
    print(r)
cur.close()

(0, 'We formed our partnership with OpenAI around a shared ambition to \\ responsibly advance cutting-edge AI research and democratize AI as a new \\ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \\ In this next phase of our partnership, developers and organizations across\xa0\\ industries will have access to the best AI infrastructure, models, and toolchain \\ with Azure to build and run their applications.', [-0.026725545525550842, -0.054436445236206055, -0.047934334725141525, 0.027443425729870796, 0.09098034352064133, -0.013835313729941845, -0.005485240835696459, -0.04992423579096794, 0.04061346501111984, 0.009536135010421276, -0.061506278812885284, -0.0077025918290019035, 0.032477278262376785, 0.017658721655607224, 0.0651795044541359, 0.14944639801979065, 0.008546381257474422, -0.029865946620702744, -0.09027165174484253, -0.010371614247560501, -0.037613723427057266, 0.028216414153575897, -0.003459315747022629, -0.056050460785627365, 0.03761869668960571, 0.0

In [14]:
def get_embedding_for_user_query(user_query):
    user_embeddings=model.encode([user_query])
    return(user_embeddings[0])
user_query_embedding=get_embedding_for_user_query("who is the founder of microsoft ?").tolist()
user_query_embedding

[-0.013981768861413002,
 0.01555590983480215,
 -0.028679419308900833,
 0.0012960012536495924,
 -0.054564740508794785,
 -0.01876084692776203,
 0.09692225605249405,
 -0.0142275495454669,
 0.012105065397918224,
 0.07103098183870316,
 -0.06160694733262062,
 0.046435970813035965,
 0.003461451269686222,
 0.00639544241130352,
 -0.09689134359359741,
 0.06958848237991333,
 -0.08100834488868713,
 -0.027593588456511497,
 -0.018633583560585976,
 -0.016732024028897285,
 -0.05927065759897232,
 -0.030028823763132095,
 -0.09232079982757568,
 -0.04062577709555626,
 0.0962909385561943,
 0.05480753630399704,
 0.00693181436508894,
 -0.02115810103714466,
 0.04490765929222107,
 -0.026880227029323578,
 -0.04038254916667938,
 -0.07271648943424225,
 0.09559698402881622,
 0.06353525817394257,
 -0.006124694831669331,
 -0.0319574773311615,
 0.1240902841091156,
 0.02192084677517414,
 -0.031090952455997467,
 -0.017874013632535934,
 0.007187938317656517,
 0.014800362288951874,
 0.015366404317319393,
 0.0382461771368

In [15]:
len(user_query_embedding)

384

In [16]:
from scipy.spatial.distance import cosine

cur = conn.cursor()
cur.execute("""SELECT text, embedding_vector FROM items""")
for r in cur.fetchall():
    text = r[0]
    embedding_vector = r[1]
    similarity = 1 - cosine(embedding_vector, user_query_embedding)
    print(f"Text: {text}, Similarity: {similarity}")
cur.close()


Text: We formed our partnership with OpenAI around a shared ambition to \ responsibly advance cutting-edge AI research and democratize AI as a new \ technology platform,” said Satya Nadella, Chairman and CEO, Microsoft. \ In this next phase of our partnership, developers and organizations across \ industries will have access to the best AI infrastructure, models, and toolchain \ with Azure to build and run their applications., Similarity: 0.3555344234782667
